In [ ]:
!pip install snowflake-snowpark-python scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
import snowflake.snowpark as super
from snowflake.snowpark import Session
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#defining my snowflakes connection parameters
connection_parameters = {
    "account":"fjdpzmq-ms56557",
    "user": "ELTONDLAMINI",
    "password": "31T0n@wipro",
    "warehouse": "LOAN_APPLICATION_SYSTEM",
    "database": "LOAN_APPLICATION_DATABASE",
    "schema": "public",
}

#Create a Snowflake session
session = Session.builder.configs(connection_parameters).create()

In [ ]:
#Read the table into a Snowpark DataFrame
snowpark_df =session.table("LOAN_T")

#Optionally, show the DataFrame
snowpark_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"LOAN_ID"  |"GENDER"  |"MARRIED"  |"DEPENDENTS"  |"EDUCATION"   |"SELF_EMPLOYED"  |"APPLICATIONINCOME"  |"COAPPLICATIONINCOME"  |"LOANAMOUNT"  |"LOAN_AMOUNT_TERM"  |"CREDIT_HISTORY"  |"PROPERT_AREA"  |"LOAN_STATUS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|LP001003   |Male      |True       |1             |Graduate      |False            |4583.0               |1508.0000000000000     |128.0         |360.0               |1.0               |Rural           |False          |
|LP001005   |Male      |True       |0             |Graduate      |True             |3000.0               |0E-13             

In [ ]:
#Convert to Pandas DataFrame
df=snowpark_df.to_pandas()
df.head(5)

,LOAN_ID,GENDER,MARRIED,DEPENDENTS,EDUCATION,SELF_EMPLOYED,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM,CREDIT_HISTORY,PROPERT_AREA,LOAN_STATUS
0,LP001003,Male,True,1,Graduate,False,4583.0,1508.0,128.0,360.0,1.0,Rural,False
1,LP001005,Male,True,0,Graduate,True,3000.0,0.0,66.0,360.0,1.0,Urban,True
2,LP001006,Male,True,0,Not Graduate,False,2583.0,2358.0,120.0,360.0,1.0,Urban,True
3,LP001008,Male,False,0,Graduate,False,6000.0,0.0,141.0,360.0,1.0,Urban,True
4,LP001013,Male,True,0,Not Graduate,False,2333.0,1516.0,95.0,360.0,1.0,Urban,True


In [ ]:
df.dtypes

LOAN_ID                 object
GENDER                  object
MARRIED                   bool
DEPENDENTS              object
EDUCATION               object
SELF_EMPLOYED           object
APPLICATIONINCOME      float64
COAPPLICATIONINCOME    float64
LOANAMOUNT             float64
LOAN_AMOUNT_TERM       float64
CREDIT_HISTORY         float64
PROPERT_AREA            object
LOAN_STATUS               bool
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LOAN_ID              381 non-null    object 
 1   GENDER               376 non-null    object 
 2   MARRIED              381 non-null    bool   
 3   DEPENDENTS           373 non-null    object 
 4   EDUCATION            375 non-null    object 
 5   SELF_EMPLOYED        360 non-null    object 
 6   APPLICATIONINCOME    369 non-null    float64
 7   COAPPLICATIONINCOME  363 non-null    float64
 8   LOANAMOUNT           373 non-null    float64
 9   LOAN_AMOUNT_TERM     370 non-null    float64
 10  CREDIT_HISTORY       351 non-null    float64
 11  PROPERT_AREA         381 non-null    object 
 12  LOAN_STATUS          381 non-null    bool   
dtypes: bool(2), float64(5), object(6)
memory usage: 33.6+ KB


In [ ]:
df.isnull().sum()

LOAN_ID                 0
GENDER                  5
MARRIED                 0
DEPENDENTS              8
EDUCATION               6
SELF_EMPLOYED          21
APPLICATIONINCOME      12
COAPPLICATIONINCOME    18
LOANAMOUNT              8
LOAN_AMOUNT_TERM       11
CREDIT_HISTORY         30
PROPERT_AREA            0
LOAN_STATUS             0
dtype: int64

In [ ]:
# Convert the object columns to numeric, forcing errors to NaNs
df['DEPENDENTS'] = pd.to_numeric(df['DEPENDENTS'], errors='coerce')

In [ ]:
df.dtypes

LOAN_ID                 object
GENDER                  object
MARRIED                   bool
DEPENDENTS             float64
EDUCATION               object
SELF_EMPLOYED           object
APPLICATIONINCOME      float64
COAPPLICATIONINCOME    float64
LOANAMOUNT             float64
LOAN_AMOUNT_TERM       float64
CREDIT_HISTORY         float64
PROPERT_AREA            object
LOAN_STATUS               bool
dtype: object

In [ ]:
df.describe()

,DEPENDENTS,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM,CREDIT_HISTORY
count,345.000000,369.000000,363.000000,373.000000,370.000000,351.000000
mean,0.492754,3563.422764,1267.005289,104.914209,340.864865,0.837607
std,0.770518,1427.371257,2388.048316,28.484822,68.549257,0.369338
min,0.000000,150.000000,0.000000,9.000000,12.000000,0.000000
25%,0.000000,2583.000000,0.000000,90.000000,360.000000,1.000000
50%,0.000000,3326.000000,830.000000,110.000000,360.000000,1.000000
75%,1.000000,4226.000000,2008.000000,127.000000,360.000000,1.000000
max,2.000000,9703.000000,33837.000000,150.000000,480.000000,1.000000


In [ ]:
# Select numerical columns
numerical_cols = df.select_dtypes(include=[np.number])
# Compute the mean of each numerical column
means = numerical_cols.mean()
# Replace null values in each numerical column with its mean
df[numerical_cols.columns] = numerical_cols.apply(lambda x: x.fillna(x.mean()))

In [ ]:
# Replace null values with mode for each column(catagorical data)
for column in df.columns:
    mode_value = df[column].mode()[0]  # Calculate mode for the column
    df[column].fillna(mode_value, inplace=True)  # Fill NaN values with mode

In [ ]:
df.isnull().sum()

LOAN_ID                0
GENDER                 0
MARRIED                0
DEPENDENTS             0
EDUCATION              0
SELF_EMPLOYED          0
APPLICATIONINCOME      0
COAPPLICATIONINCOME    0
LOANAMOUNT             0
LOAN_AMOUNT_TERM       0
CREDIT_HISTORY         0
PROPERT_AREA           0
LOAN_STATUS            0
dtype: int64

In [ ]:
# Convert categorical variables to numeric
label_encoders = {}
for column in ['GENDER', 'MARRIED', 'EDUCATION', 'SELF_EMPLOYED', 'PROPERT_AREA', 'LOAN_STATUS']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Handle 'Dependents' separately if it has '3+' as a category
df['DEPENDENTS'] = df['DEPENDENTS'].replace('3+', 3).astype(int)

In [ ]:
# Select numerical columns for outlier fixing
numerical_cols = ['APPLICATIONINCOME', 'COAPPLICATIONINCOME', 'LOANAMOUNT', 'LOAN_AMOUNT_TERM']

# Calculate IQR for each column
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# Define outliers using IQR rule
outliers_lower = df[numerical_cols] < (Q1 - 1.5 * IQR)
outliers_upper = df[numerical_cols] > (Q3 + 1.5 * IQR)

# Fix outliers by replacing with median
for col in numerical_cols:
    median = df[col].median()
    df.loc[outliers_lower[col], col] = median
    df.loc[outliers_upper[col], col] = median

# Display updated data with fixed outliers
print("Data after fixing outliers with median:")
df[numerical_cols].describe()

Data after fixing outliers with median:


,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM
count,381.000000,381.000000,381.000000,381.0
mean,3400.105179,1053.052533,106.510010,360.0
std,1073.465086,1133.778069,25.848411,0.0
min,645.000000,0.000000,35.000000,360.0
25%,2600.000000,0.000000,94.000000,360.0
50%,3357.000000,1041.000000,110.000000,360.0
75%,3941.000000,1857.000000,127.000000,360.0
max,6500.000000,4490.000000,150.000000,360.0


In [ ]:
df.head(5)

,LOAN_ID,GENDER,MARRIED,DEPENDENTS,EDUCATION,SELF_EMPLOYED,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM,CREDIT_HISTORY,PROPERT_AREA,LOAN_STATUS
0,LP001003,1,1,1,0,0,4583.0,1508.0,128.0,360.0,1.0,0,0
1,LP001005,1,1,0,0,1,3000.0,0.0,66.0,360.0,1.0,2,1
2,LP001006,1,1,0,1,0,2583.0,2358.0,120.0,360.0,1.0,2,1
3,LP001008,1,0,0,0,0,6000.0,0.0,141.0,360.0,1.0,2,1
4,LP001013,1,1,0,1,0,2333.0,1516.0,95.0,360.0,1.0,2,1


In [ ]:
X = df.drop(columns=['LOAN_ID', 'LOAN_STATUS'])
y = df['LOAN_STATUS']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = RandomForestClassifier(n_estimators=381, random_state=42)
model.fit(X_train, y_train)
#model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=2)

RandomForestClassifier(n_estimators=381, random_state=42)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))
#print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8173913043478261


In [ ]:
# Convert predictions back to original encoding
y_pred_labels = label_encoders['LOAN_STATUS'].inverse_transform(y_pred)

# Create a DataFrame for predictions
predicted_df = df.iloc[y_test.index].copy()
predicted_df['PREDICTED_LOAN_STATUS'] = y_pred_labels

In [ ]:
predicted_df

,LOAN_ID,GENDER,MARRIED,DEPENDENTS,EDUCATION,SELF_EMPLOYED,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM,CREDIT_HISTORY,PROPERT_AREA,LOAN_STATUS,PREDICTED_LOAN_STATUS
266,LP002348,1,1,0,0,0,5829.0,0.0,138.0,360.0,1.0,0,1,True
192,LP001977,1,1,1,0,0,1625.0,1803.0,96.0,360.0,1.0,2,1,True
46,LP001206,1,1,0,0,0,3029.0,0.0,99.0,360.0,1.0,2,1,True
55,LP001259,1,1,1,0,1,1000.0,3022.0,110.0,360.0,1.0,2,0,True
57,LP001265,0,0,0,0,0,3846.0,0.0,111.0,360.0,1.0,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,LP001068,1,1,0,0,0,2799.0,2253.0,122.0,360.0,1.0,1,1,True
157,LP001836,0,0,2,0,0,3427.0,0.0,138.0,360.0,1.0,2,0,True
24,LP001109,1,1,0,0,0,1828.0,1330.0,100.0,360.0,0.0,2,0,False
116,LP001658,1,0,0,0,0,3858.0,0.0,76.0,360.0,1.0,1,1,True


In [ ]:
#Convert back to categorical variables
for column in ['GENDER', 'MARRIED', 'EDUCATION', 'SELF_EMPLOYED', 'PROPERT_AREA', 'LOAN_STATUS']:
    predicted_df[column] = label_encoders[column].inverse_transform(predicted_df[column])

In [ ]:
predicted_df.head(5)

,LOAN_ID,GENDER,MARRIED,DEPENDENTS,EDUCATION,SELF_EMPLOYED,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM,CREDIT_HISTORY,PROPERT_AREA,LOAN_STATUS,PREDICTED_LOAN_STATUS
266,LP002348,1,1,0,0,0,5829.0,0.0,138.0,360.0,1.0,Rural,True,True
192,LP001977,1,1,1,0,0,1625.0,1803.0,96.0,360.0,1.0,Urban,True,True
46,LP001206,1,1,0,0,0,3029.0,0.0,99.0,360.0,1.0,Urban,True,True
55,LP001259,1,1,1,0,1,1000.0,3022.0,110.0,360.0,1.0,Urban,False,True
57,LP001265,0,0,0,0,0,3846.0,0.0,111.0,360.0,1.0,Semiurban,True,True


In [ ]:
# Feature engineering
predicted_df['TOTALINCOME'] = predicted_df['APPLICATIONINCOME'] + predicted_df['COAPPLICATIONINCOME']

# Select features and target
features = ['APPLICATIONINCOME', 'COAPPLICATIONINCOME', 'LOAN_AMOUNT_TERM', 'CREDIT_HISTORY']
X = predicted_df[features]
y = predicted_df['LOANAMOUNT']  # Target variable is LoanAmount

# Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the entire dataset
loan_amount_predictions = model.predict(X)

# Add predictions to the original dataframe
predicted_df['LOAN_AMOUNT_PREDICTIONS'] = loan_amount_predictions

In [ ]:
predicted_df.tail(5)

,LOAN_ID,GENDER,MARRIED,DEPENDENTS,EDUCATION,SELF_EMPLOYED,APPLICATIONINCOME,COAPPLICATIONINCOME,LOANAMOUNT,LOAN_AMOUNT_TERM,CREDIT_HISTORY,PROPERT_AREA,LOAN_STATUS,PREDICTED_LOAN_STATUS,TOTALINCOME,LOAN_AMOUNT_PREDICTIONS
17,LP001068,1,1,0,0,0,2799.0,2253.0,122.0,360.0,1.0,Semiurban,True,True,5052.0,120.678284
157,LP001836,0,0,2,0,0,3427.0,0.0,138.0,360.0,1.0,Urban,False,True,3427.0,122.685714
24,LP001109,1,1,0,0,0,1828.0,1330.0,100.0,360.0,0.0,Urban,False,False,3158.0,98.148284
116,LP001658,1,0,0,0,0,3858.0,0.0,76.0,360.0,1.0,Semiurban,True,True,3858.0,103.042500
333,LP002714,1,0,1,1,0,2679.0,1302.0,94.0,360.0,1.0,Semiurban,True,True,3981.0,96.240000


In [ ]:
#Convert pandas DataFrame to snowpark DataFrame
snowpark_prediction_df = session.create_dataframe(predicted_df)

/usr/local/lib/python3.10/dist-packages/snowflake/snowpark/session.py:2372: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


In [ ]:
#write the prediction back to a snowflakes table
snowpark_prediction_df.write.save_as_table("Pridicted_tables",mode="overwrite")

In [ ]:
#close the session
session.close()